# Load Data

In [1]:
from proj1_helpers import *
from implementations import *
from cross_validation import *
import numpy as np

In [2]:
y, tx, ids = load_csv_data('Data/train.csv', False)
test_y, test_tx, test_ids= load_csv_data('Data/test.csv', False)

# Preprocessing

In [3]:
print(test_y.shape)

(568238,)


# Cross-Validation

In [4]:
from cross_validation import *
cross_validation_run(tx, y)

0  loss train: 0.339877979948  loss test 0.339177848533  weights: [  8.24365242e-05  -7.24862238e-03  -6.02663292e-03  -5.84495473e-04
  -1.60237804e-02   4.59970849e-04  -2.58226584e-02   3.24571114e-01
  -6.82270989e-05   4.69521976e-01  -2.20071164e-01   9.32217557e-02
   5.97881697e-02  -4.61270935e-01  -1.11205566e-03  -1.76979966e-03
  -4.56646244e-01   7.01557760e-04   1.05571491e-03   3.73326831e-03
   9.08484900e-04  -5.37216335e-04  -3.27802339e-01  -1.36657184e-03
   5.97278301e-04   1.21345129e-03  -1.80785276e-03  -6.20051619e-03
  -9.93959085e-03  -4.69033367e-01]
1  loss train: 0.339331811085  loss test 0.340835437808  weights: [  7.95272654e-05  -7.23002152e-03  -6.06689176e-03  -5.61374695e-04
  -2.46136572e-02   4.91058255e-04  -2.60313943e-02   3.22924535e-01
  -9.09906124e-05  -3.45446340e+00  -2.20277478e-01   9.61767883e-02
   6.86923218e-02   3.46284299e+00   8.91034938e-04  -4.27450854e-04
   3.46764864e+00  -2.37292664e-03   1.61636966e-03   3.63502290e-03
   1

# Weight Computation

In [7]:
w, loss = least_squares(y,tx)
print(w, loss)

0.339686809477 [  8.03494315e-05  -7.20202273e-03  -6.05417273e-03  -5.47559066e-04
  -1.93874702e-02   4.73451621e-04  -2.60379054e-02   3.25106300e-01
  -3.80780260e-05  -2.72729930e+00  -2.21220140e-01   9.50794092e-02
   6.40351616e-02   2.73555898e+00  -3.31801229e-04  -9.54325120e-04
   2.74031573e+00  -5.34164922e-04   9.73498609e-04   3.69225052e-03
   3.54487428e-04  -5.43344599e-04  -3.30448035e-01  -1.40800497e-03
   8.31432882e-04   1.02117272e-03  -1.68047416e-03  -5.83664815e-03
  -1.11087997e-02   2.72775924e+00]


# Output Data

In [ ]:
name = 'first-submission'
y_pred = predict_labels(w,test_tx)
print(y_pred)


create_csv_submission(test_ids, y_pred, name)